In [1]:
## universal segmentor, run notebook before training

In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from tqdm import tqdm
import tifffile as tif
import os
import pickle
import random

In [3]:
import elasticdeform
import albumentations
import Augmentor

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from stardist import random_label_cmap
np.random.seed(6)
lbl_cmap = random_label_cmap()

In [5]:
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

## functions

In [6]:
def load_batch(IMG_path,MASK_path,start_id,batch_size,segmaps_per_img = 1):
    # loads first image
    image = imageio.imread(IMG_path[0])
    mask = imageio.imread(MASK_path[0])
    im_names = []
    img_batch = np.zeros((batch_size,*image.shape),dtype = image.dtype)
    mask_batch = np.zeros((batch_size,*mask.shape,segmaps_per_img),dtype = mask.dtype)
    for i in range(0,batch_size):
        img_batch[i,:,:] = imageio.imread(IMG_path[start_id+i])
        mask_batch[i,:,:,0] = imageio.imread(MASK_path[start_id+i])
        im_names.append(os.path.split(IMG_path[start_id+i])[1])
    return img_batch,mask_batch,im_names

In [7]:
def save_batch(img_aug,mask_aug,names_new,path):
    # saves a batch of augmented images to the new directory
    for i, im in enumerate(img_aug):
        im_name = os.path.join(path,"images",names_new[i])
        print("Saving im_name..", im_name)
        imageio.imwrite(im_name,im)
        mask_name = os.path.join(path,"masks",names_new[i])
        imageio.imwrite(mask_name,mask_aug[i])
    return

In [8]:
def augment_batch(im,ma,names,SD,aug_per_img,itter):
    '''creates random augmentations
       based on predefined sets handed over as SD
       aug_per_img defines the number of different augmentation tpyes per image
       itter the augmentation per image per pipeline
    '''
    images_aug = []
    masks_aug = []
    names_aug = []
    for i,image in enumerate(im):
        mask = ma[i,:,:,:]
        segmap = SegmentationMapsOnImage(mask, shape=image.shape)
        AugList = random.sample(range(0,len(SD)),aug_per_img)
        for aug_id in AugList:
            pipeline = SD[aug_id]
            for it in range(0,itter):
                images_aug_i, segmaps_aug_i = pipeline(image=image, segmentation_maps=segmap)
                images_aug.append(images_aug_i)
                masks_aug.append(segmaps_aug_i.get_arr())
                names_aug.append(names[i].replace(".tif","_{}_{}.tif".format(str(aug_id).zfill(4),str(it).zfill(3))))
    return images_aug,masks_aug,names_aug

## variables

In [9]:
# do not change! it defines if augmntations are applied only to the imade (e.g. noise)
# or to image and mask (e.g) distortiions
augment_core_setting = {}
augment_core_setting["add noise"] = False # not applied to mask
augment_core_setting["rotate_flip"] = True # applied to mask
augment_core_setting["zoom"] = True
augment_core_setting["elastic_deform"] = True
augment_core_setting["simulate_uneven_illumination"] = True

In [10]:
# define how to augment (chance each time for a pipeline)
augment = True
augment_setting = {}
augment_setting["new name"] = "SD_dsb2018_Aug001"
augment_setting["add noise"] = False
augment_setting["rotate_flip"] = False
augment_setting["zoom"] = False
augment_setting["elastic_deform"] = False
augment_setting["elastic_setting"] = {}
augment_setting["simulate_uneven_illumination"] = False
augment_setting["batch_size"] = 60
if augment_setting["elastic_deform"]:
    augment_setting["elastic_setting"]["points"] = 3
    augment_setting["elastic_setting"]["itteration"] = 5 

In [11]:
# define location of data
trainModelSettings = {}
trainModelSettings["root"] = "/mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT"
trainModelSettings["data"] = "SD_dsb2018"
trainModelSettings["path"] = os.path.join(trainModelSettings["root"],trainModelSettings["data"])
trainModelSettings["path_out"] =  os.path.join(trainModelSettings["root"],augment_setting["new name"])

In [12]:
file_path = "./my_runs/augment_settings_xl.pkl"
infile = open(file_path,'rb')
parameter = pickle.load(infile)
print("Loading processing pipeline from",file_path)
infile.close()
seqD = parameter[0]

Loading processing pipeline from ./my_runs/augment_settings_xl.pkl


In [13]:
#  save settings
parameter = [augment_core_setting,augment_setting,trainModelSettings,seqD]

# save it
file_name = "./my_runs/Parameter_Augment_{}.pkl".format(augment_setting["new name"])
file_name_load = "./Train/Augment/my_runs/Parameter_Augment_{}.pkl".format(augment_setting["new name"])
print("The parameters for augmentation were stored in this file: \n",file_name_load)
outfile = open(file_name,'wb')
pickle.dump(parameter,outfile)
outfile.close()

The parameters for augmentation were stored in this file: 
 ./Train/Augment/my_runs/Parameter_Augment_SD_dsb2018_Aug001.pkl


## main code

In [14]:
IMG_path = sorted(glob('{}/train/images/*.tif'.format(trainModelSettings["path"])))
MASK_path = sorted(glob('{}/train/masks/*.tif'.format(trainModelSettings["path"])))

In [15]:
assert all(os.path.split(IMG_path[x])[1]==os.path.split(MASK_path[x])[1] for x in range(0,len(IMG_path)))

In [16]:
start_id = 0
X,Y,N = load_batch(IMG_path,MASK_path,start_id,augment_setting["batch_size"])

In [17]:
Xnew,Ynew,Nnew = augment_batch(X,Y,N,seqD,2,3)

In [18]:
save_batch(Xnew,Ynew,Nnew,trainModelSettings["path_out"])

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/07fb37aafa6626608af90c1e18f6a743f29b6b233d2e427dcd1102df6a916cf5_0002_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/07fb37aafa6626608af90c1e18f6a743f29b6b233d2e427dcd1102df6a916cf5_0002_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/07fb37aafa6626608af90c1e18f6a743f29b6b233d2e427dcd1102df6a916cf5_0002_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/07fb37aafa6626608af90c1e18f6a743f29b6b233d2e427dcd1102df6a916cf5_0000_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/07fb37aafa6626608af90c1e18f6a743f29b6b233d2e427dcd1102df6a916cf5_0000_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/07fb37aafa6626608af90c1e18f6a743f29b6b233d2e427dcd1102df6a

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/1f6b7cead15344593b32d5f2345fc26713dc74d9b31306c824209d67da401fd8_0005_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/1f6b7cead15344593b32d5f2345fc26713dc74d9b31306c824209d67da401fd8_0005_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/1f6b7cead15344593b32d5f2345fc26713dc74d9b31306c824209d67da401fd8_0002_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/1f6b7cead15344593b32d5f2345fc26713dc74d9b31306c824209d67da401fd8_0002_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/1f6b7cead15344593b32d5f2345fc26713dc74d9b31306c824209d67da401fd8_0002_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/212b858a66f0d23768b8e3e1357704fc2f4cf4bbe7eed8cd59b5d01031

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/449a9c32e53a37c8a86e01c199155c8da3958b631088e10f6fe43c2119defe51_0004_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/449a9c32e53a37c8a86e01c199155c8da3958b631088e10f6fe43c2119defe51_0004_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/449a9c32e53a37c8a86e01c199155c8da3958b631088e10f6fe43c2119defe51_0004_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/449f41710769584b5e4eca8ecb4c76d5272605f27da2949e6285de0860d2cbc0_0005_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/449f41710769584b5e4eca8ecb4c76d5272605f27da2949e6285de0860d2cbc0_0005_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/449f41710769584b5e4eca8ecb4c76d5272605f27da2949e6285de0860

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/4ee5850b63549794eb3ecd3d5f5673164ac16936e36ecc3700da886e3b616149_0000_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/53ad09e4348767bece0165884bf40c10b72ae18444e3f414a850442f02385efc_0002_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/53ad09e4348767bece0165884bf40c10b72ae18444e3f414a850442f02385efc_0002_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/53ad09e4348767bece0165884bf40c10b72ae18444e3f414a850442f02385efc_0002_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/53ad09e4348767bece0165884bf40c10b72ae18444e3f414a850442f02385efc_0005_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/53ad09e4348767bece0165884bf40c10b72ae18444e3f414a850442f02

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/6f8197baf738986a1ec3b6ba92b567863d897a739376b7cec5599ad6cecafdfc_0001_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/6fe2df6de1d962b90146c822bcefc84d0d3d6926fdfbacd3acdc9de830ee5622_0003_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/6fe2df6de1d962b90146c822bcefc84d0d3d6926fdfbacd3acdc9de830ee5622_0003_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/6fe2df6de1d962b90146c822bcefc84d0d3d6926fdfbacd3acdc9de830ee5622_0003_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/6fe2df6de1d962b90146c822bcefc84d0d3d6926fdfbacd3acdc9de830ee5622_0007_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/6fe2df6de1d962b90146c822bcefc84d0d3d6926fdfbacd3acdc9de830

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/8d29c5a03e0560c8f9338e8eb7bccf47930149c8173f9ba4b9279fb87d86cf6d_0007_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177ef25a9dee25ffe8842_0005_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177ef25a9dee25ffe8842_0005_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177ef25a9dee25ffe8842_0005_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177ef25a9dee25ffe8842_0002_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/8efed2e62c919e6d70a2ab548b1a33014877fe8a23f177ef25a9dee25f

Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/a891bbc89143bca7a717386144eb061ec2d599cba24681389bcb3a2fedb8ff8c_0002_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/a90cad45551d62c5cfa89517df8eb5e8f2f87f1a6e6678e606907afcbad91731_0002_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/a90cad45551d62c5cfa89517df8eb5e8f2f87f1a6e6678e606907afcbad91731_0002_001.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/a90cad45551d62c5cfa89517df8eb5e8f2f87f1a6e6678e606907afcbad91731_0002_002.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/a90cad45551d62c5cfa89517df8eb5e8f2f87f1a6e6678e606907afcbad91731_0004_000.tif
Saving im_name.. /mnt/ag-microscopy/SampleDataML/forOpsef_Final/GT/SD_dsb2018_Aug001/images/a90cad45551d62c5cfa89517df8eb5e8f2f87f1a6e6678e606907afcba